# Recurring Events

Expressing recurring events (meetings, alarms, tasks) is a fairly common task, but is complicated by the realities of the calendar system and many of the other things we've discussed in the tutorial so far. The closest thing to a standard way to express recurring events is defined in the iCalendar Spec: [RFC 5545](https://tools.ietf.org/html/rfc5545): "Internet Calendaring and Scheduling Core Object Specification".

![RFC 5545](images/RFC5545-toc.png)

The iCalendar spec is a long document with a nested hierarchy of objects, calendars, events, alarms, TODOs, etc. Buried in that definition in section 3.3.10 is the definition of the *recurrence rule* data type for expressing recurring events.

`dateutil` provides an implementation of this data type in the `rrule` module (partially compliant with RFC5545 and its amendments, originally written to be compliant with RFC 2445). In this section, we will cover how to use the `dateutil.rrule` module.

### RRULE components

The fundamental elements of an `rrule` are:

- `dtstart`: The start point of the recurrence (this is similar to a phase)
- `freq`: The units of the fundamental frequency of the recurrence. It takes the values `YEARLY`, `MONTHLY`, `WEEKLY`, `DAILY`, `HOURLY`, `MINUTELY`, `SECONDLY`
- `interval`: The fundamental frequency of the recurrence, in units of `freq`. If unspecified, this is 1.

To see how these affect the rrule, we'll play around with an `HOURLY` rrule.

In [ ]:
from datetime import datetime

from dateutil.rrule import rrule
from dateutil.rrule import MO, TU, WE, TH, FR, SA, SU
from dateutil.rrule import YEARLY, MONTHLY, WEEKLY, DAILY, HOURLY, MINUTELY, SECONDLY

from helper_functions import print_dtlist

In [ ]:
hourly = rrule(freq=HOURLY, interval=1, dtstart=datetime(2016, 7, 18, 9), count=3)
print_dtlist(hourly)

In [ ]:
dtstart_rr = hourly.replace(dtstart=datetime(2016, 7, 18, 10))
print_dtlist(dtstart_rr)

In [ ]:
interval_2 = hourly.replace(interval=2)
print_dtlist(interval_2)

### Limiting RRULEs
An `rrule` starts at `dtstart` (it cannot run backwards), and by default will generate dates indefinitely (in practice, limited by the point at which the `datetime` type cannot represent the dates it would return). There are two, mutually exclusive ways to set limits on the rules:

- `count`: A total number of valid instances to generate
- `until`: The maximum `datetime` value that can be emitted by the `rrule`

### Retrieving subsets
A fairly common operation is to retrieve a subset of the valid recurrences of an `rrule` based on a date. ``rule`` exposes several methods to achieve this:

- `after`: Retrieve the *first* recurrence of the rule *after* the provided datetime
- `before`: Retrieve the *last* recurrence of the rule *before* the provided datetime
- `between`: Retrieve all recurrences of the rule between two datetimes

For example:

In [ ]:
rr = rrule(freq=MONTHLY, interval=4, dtstart=datetime(1995, 7, 10))

In [ ]:
rr.after(datetime(2015, 2, 1))

In [ ]:
rr.before(datetime(1997, 1, 1))

In [ ]:
rr.between(datetime(2001, 1, 1), datetime(2002, 1, 1))

### Exercise: Play around with some rrules
Before we get into more complicated rules, take a few minutes to practice creating `rrule`s and using them.

Examples:

- Every day at 13:30
- Every other week for 6 weeks (try this with both `count` and `until`)
- What happens when you call `after` with a datetime after the last recurrence? What about `before`?
- How does `between` interact with `count` and `until`?

## `byxxx` rules

So far, we haven't seen any rules that couldn't be implemented (fairly) easily by applying `relativedelta`s to a base date, but we often want to express more common recurrences, like "the third Friday of every month" or "the 1st of each month at 12:30", for these, we need to bring in "by" rules.

`byxxx` rules serve to modify the frequency of the recurrence in some way. The supported rules are `bymonth`, `bymonthday`, `byyearday`, `byweekno`, `byweekday`, `byhour`, `byminute` and `bysecond`, `bysetpos` and `byeaster`.

- When the `byxxx` rule specifies a component *greater than or equal to `freq`*, it is a constraint, and (generally) will *reduce* the frequency of the occurrence. For example:

In [ ]:
# Base frequency is DAILY, but restricted to Tuesdays in November
rr = rrule(freq=DAILY, bymonth=11, byweekday=(TU, ),
           dtstart=datetime(2015, 1, 1, 12), count=5)

print_dtlist(rr)

- When the `byxxx` rule is *less than* the frequency, it will generally *increase* the frequency of the recurrence:

In [ ]:
# On the 1st, 15th and 30th of every month
rr = rrule(freq=MONTHLY, bymonthday=(1, 15, 30),
           dtstart=datetime(2015, 1, 16, 12, 15), count=4)

print_dtlist(rr)

### `byweekday`
Similar to weekday constants in the `relativedelta` module, `rrule` also allows specifying rules by weekday. In this case the arguments to the weekday constants are interpreted as the index of all weekdays in one period of the `freq`.

So, for example, if you specify `byweekday=(FR(1), FR(-1))`, with a `MONTHLY` rule, you can get the first and last Friday of the month:

In [ ]:
rr = rrule(freq=MONTHLY, byweekday=(FR(1), FR(-1)),
           dtstart=datetime(2021, 1, 1), count=4)

print_dtlist(rr)

And if you make it a `YEARLY`, you get the first and last Friday of the *year*:

In [ ]:
rr_y = rr.replace(freq=YEARLY)
print_dtlist(rr_y)

For any `freq` smaller than a month, the arguments to the weekday are ignored, and you simply get all Fridays:

In [ ]:
rr_w = rr.replace(freq=WEEKLY)
print_dtlist(rr_w)

### Exercise: Martin Luther King Day

[Martin Luther King, Jr Day](https://en.wikipedia.org/wiki/Martin_Luther_King_Jr._Day) is a US holiday that occurs every year on the third Monday in January. Write a recurrence rule that genrates Martin Luther King Day, starting from its first observance in 1986.

In [ ]:
import rr_answers, rr_tests

In [ ]:
### Replace this with your own definition of MLK_DAY
mlk_day = ...

In [ ]:
import rr_tests

### Uncomment to test
# rr_tests.test_mlk_day(mlk_day)